In [2]:
import pandas as pd 
import requests
import numpy as np
import re
import warnings
from pandas import json_normalize


In [3]:
import os.path

file_exists = os.path.exists("inst_full.csv")

print(file_exists)

True


THE CELL BEOW CHECKS TO SEE IF YOU ALREADY HAVE THE DATA AND IF NOT SENDS GET REQUESTS TO THE API

NOTE - THIS DOES NOT CHECK TO ENSURE THE FILE IS CURRENT OR FREE OF ERRORS.  IF YOU WOULD LIKE TO UPDATE YOUR FILE OR YOU ARE CONCERNED THAT YOUR FILE HAS ERRORS, SIMPLY DELETE YOUR FILE AND A NEW, UPDATED FILE WILL BE CREATED.

KNOWLEDGE CHECK 2 - See the last 20 lines in the cell below (most of the code in the ELSE statement).  This cleans the data by checking for columns with mixed data types and converting the entire column to strings.  It converts the data type, but also displays an error to identify the affected columns.  This is a solution I found on stackoverflow that I wanted to implement in my code.

In [4]:

import pandas


if file_exists == False:
    print("FILE DOES NOT EXIST")

    response_API = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=0')
    #print(response_API.status_code)

    response_API2 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=10000')
    #print(response_API2.status_code)

    response_API3 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=20000')
    #print(response_API3.status_code)

    print("Get request sucessful from API")


    data = response_API.json()
    #print(type(data))

    data2 = response_API2.json()
    #print(type(data2))

    data3 = response_API3.json()
    #print(type(data3))



    institution_df = json_normalize(data['data'])
    #print(institution_df.head())
    #institution_df.to_csv('inst.csv')

    institution_df2 = json_normalize(data2['data'])
    #print(institution_df2.head())
    #institution_df2.to_csv('inst2.csv')

    institution_df3 = json_normalize(data3['data'])
    #print(institution_df3.head())
    #institution_df3.to_csv('inst3.csv')
    
    print("Data from API converted to json")


    institution_df = institution_df.reindex(sorted(institution_df.columns), axis=1)
    institution_df2 = institution_df2.reindex(sorted(institution_df2.columns), axis=1)
    institution_df3 = institution_df3.reindex(sorted(institution_df3.columns), axis=1)



    full_data = [institution_df, institution_df2, institution_df3]
    df = pd.concat(full_data)
    df.columns = df.columns.str.lstrip("data.")
    df.to_csv('inst_full.csv')
    
    print("New file created, shape should be (27816, 168)")
    print(df.shape)
    

else:
    print("FILE ALREADY EXISTS")

    myfile = 'inst_full.csv'
    target_type = str  # The desired output type

    with warnings.catch_warnings(record=True) as ws:
        warnings.simplefilter("always")

        mydata = pandas.read_csv(myfile, sep="|", header=None)
        print("Warnings raised:", ws)
        # We have an error on specific columns, try and load them as string
        for w in ws:
            s = str(w.message)
            print("Warning message:", s)
            match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
            if match:
                columns = match.group(1).split(',') # Get columns as a list
                columns = [int(c) for c in columns]
                print("Applying %s dtype to columns:" % target_type, columns)
                mydata.iloc[:,columns] = mydata.iloc[:,columns].astype(target_type)

    df = pd.read_csv('inst_full.csv')
    

FILE ALREADY EXISTS
Warnings raised: []


C:\Users\mjlaw\AppData\Local\Temp\ipykernel_29156\3671726210.py:82: DtypeWarning: Columns (108,109,110,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('inst_full.csv')


In [5]:
#full_df = institution_df.append(institution_df.append(institution_df3))
#.append is being deprecated - use .concat instead

Knowledge Check 2 - Reformating the column names so they are all lower case and spaces are replaced with an underscore

In [6]:
#print(df.columns)
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
#print(df.columns)

In [7]:
bank_count = len(df)
print("There are", bank_count, "institutuions with data on the FDIC's API.\n")

active = df['active'].value_counts()[0]
print("There are", active, "active institutions registered with the FDIC.\n")

inactive = df['active'].value_counts()[1]
print("There are", inactive, "inactive institutions registered with the FDIC.\n")

There are 27816 institutuions with data on the FDIC's API.

There are 23098 active institutions registered with the FDIC.

There are 4718 inactive institutions registered with the FDIC.



In [8]:
active_df = df[df.active == 0]
print(len(active_df))
print(active_df.head())

23098
   unnamed:_0  active                          address address2     asset  \
0           0       0              8 Washington Street      NaN       NaN   
1           1       0                201 N.E. A Street      NaN  712714.0   
2           2       0              1701 Warwood Avenue      NaN  338215.0   
3           3       0               1 East Main Street      NaN  126549.0   
5           5       0  Broadway And West Walnut Street      NaN  227248.0   

  bkclass  callform   cb                                cbsa cbsa_div  ...  \
0      SM       NaN  NaN         Portland-South Portland, ME      NaN  ...   
1      SM      41.0  0.0  Fayetteville-Springdale-Rogers, AR      NaN  ...   
2       N      41.0  1.0                     Wheeling, WV-OH      NaN  ...   
3      SM      33.0  0.0                                 NaN      NaN  ...   
5      NM      33.0  0.0                       Green Bay, WI      NaN  ...   

   ultcert  uninum  webaddr    zip count  score  changec12  ch

In [9]:
inactive_df = df[df.active == 1]
print(len(inactive_df))
print(inactive_df.head())

4718
    unnamed:_0  active             address address2      asset bkclass  \
4            4       1       86 E Water St      NaN   144049.0      NM   
11          11       1        403 W 8th St      NaN   382346.0      NM   
12          12       1  1155 County Road D      NaN   207276.0      NM   
15          15       1       112 E Main St      NaN   150653.0      NM   
32          32       1       130 S Main St      NaN  2593020.0       N   

    callform   cb               cbsa cbsa_div  ...  ultcert  uninum  \
4       51.0  1.0                NaN      NaN  ...    10004    6394   
11      51.0  1.0        Madison, WI      NaN  ...    10011    6400   
12      51.0  1.0  Stevens Point, WI      NaN  ...    10012    6401   
15      51.0  1.0     Eau Claire, WI      NaN  ...    10015    6404   
32      51.0  1.0    Fond du Lac, WI      NaN  ...    10044    6419   

                             webaddr    zip count  score  changec12  \
4            http://www.ergobank.com  53946   NaN   

In [10]:
min = "{:,.2f}".format(active_df["asset"].min())
max = "{:,.2f}".format(active_df["asset"].max())
mean = "{:,.2f}".format(active_df["asset"].mean())

print("The minimum assets held by an active FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an active FDIC registered institution are ${} \n".format(max))
print("The average assets held by an active FDIC registered institution are ${} \n".format(mean))

The minimum assets held by an active FDIC registered institution are $1.00 

The maximum assets held by an active FDIC registered institution are $510,083,000.00 

The average assets held by an active FDIC registered institution are $787,717.14 



In [11]:
min = "{:,.2f}".format(inactive_df["asset"].min())
max = "{:,.2f}".format(inactive_df["asset"].max())
mean = "{:,.2f}".format(inactive_df["asset"].mean())

print("The minimum assets held by an inactive FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an inactive FDIC registered institution are ${} \n".format(max))
print("The average assets held by an inactive FDIC registered institution are ${} \n".format(mean))

The minimum assets held by an inactive FDIC registered institution are $3,134.00 

The maximum assets held by an inactive FDIC registered institution are $3,308,575,000.00 

The average assets held by an inactive FDIC registered institution are $4,987,805.79 



In [12]:
min = "{:,.2f}".format(df["asset"].min())
max = "{:,.2f}".format(df["asset"].max())
mean = "{:,.2f}".format(df["asset"].mean().round(2))

print("The minimum assets held by an FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an FDIC registered institution are ${} \n".format(max))
print("The average assets held by an FDIC registered institution are ${} \n".format(mean))

The minimum assets held by an FDIC registered institution are $1.00 

The maximum assets held by an FDIC registered institution are $3,308,575,000.00 

The average assets held by an FDIC registered institution are $1,618,986.53 



In [13]:
"""

Standarad deviation for later analysis

std = df["data.ASSET"].std().round(2)
top_std = (mean + std).round(2)
btm_std = (mean - std).round(2)

print(std)
print("Most FDIC registed banks have between ${} and ${} \n".format(btm_std, top_std))

"""

'\n\nStandarad deviation for later analysis\n\nstd = df["data.ASSET"].std().round(2)\ntop_std = (mean + std).round(2)\nbtm_std = (mean - std).round(2)\n\nprint(std)\nprint("Most FDIC registed banks have between ${} and ${} \n".format(btm_std, top_std))\n\n'

In [14]:
# to format as currency
#.style.format('${0:,.2f}')

In [15]:
#To add banks with FDIC violations

absolute_path = os.path.abspath('')
relative_path = "ots-enforcement-order-listing.csv"
full_path = os.path.join(absolute_path, relative_path)
enforcment_df = pd.read_csv(full_path)
print(len(enforcment_df))
print(enforcment_df.head())

3547
   Record ID Docket Number     IID  \
0          1            29    29.0   
1          2          8327  8327.0   
2          3          3916  3916.0   
3          4           NaN     NaN   
4          5           NaN     NaN   

                                   Unnamed: 3 DOCASSOCIATION Unnamed: 5  \
0  http://www.occ.gov/static/ots/enforcement/          93296       .pdf   
1  http://www.occ.gov/static/ots/enforcement/          93221       .pdf   
2  http://www.occ.gov/static/ots/enforcement/          93233       .pdf   
3  http://www.occ.gov/static/ots/enforcement/          93799       .pdf   
4  http://www.occ.gov/static/ots/enforcement/          94981       .pdf   

                                          Unnamed: 6  \
0  http://www.occ.gov/static/ots/enforcement/9329...   
1  http://www.occ.gov/static/ots/enforcement/9322...   
2  http://www.occ.gov/static/ots/enforcement/9323...   
3  http://www.occ.gov/static/ots/enforcement/9379...   
4  http://www.occ.gov/static/ots/en

In [16]:
print(enforcment_df.columns)
enforcment_df.columns= enforcment_df.columns.str.lower()
enforcment_df.columns = enforcment_df.columns.str.replace(' ', '_')
print(enforcment_df.columns)

Index(['Record ID', 'Docket Number', 'IID', 'Unnamed: 3', 'DOCASSOCIATION',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Link to Enforcement Action',
       'Order Number', 'Institution Name', 'First Name', 'Last Name', 'City',
       'State', 'OTS Region', 'Issue Date', 'Type Code', 'Type Description',
       'Type of Order', 'Notes'],
      dtype='object')
Index(['record_id', 'docket_number', 'iid', 'unnamed:_3', 'docassociation',
       'unnamed:_5', 'unnamed:_6', 'unnamed:_7', 'link_to_enforcement_action',
       'order_number', 'institution_name', 'first_name', 'last_name', 'city',
       'state', 'ots_region', 'issue_date', 'type_code', 'type_description',
       'type_of_order', 'notes'],
      dtype='object')


In [17]:
#for d in enforcment_df['docket_number'].astype(str):
#    if enforcment_df['docket_number'].str.startswith('H'):
#        enforcment_df['docket_number'].str.replace(' ', '')

Knowledge Check 2 - The next two cells remove improperly formatted docket numbers and then remove rows without docket numbers from the data frame

In [18]:
enforcment_df['docket_number'] = np.where(enforcment_df['docket_number'].str.startswith('H'), '', enforcment_df['docket_number'])

In [19]:
#create a new df so the old df is unaltered

#sorts the new df by docket number
enf_df1 = enforcment_df.sort_values('docket_number')

#converts emtpy strings to NaN and then drops rows that do not have a Docket Number
enf_df1['docket_number'].replace('', np.nan, inplace=True)
enf_df1.dropna(subset=['docket_number'], inplace=True)

#Gives the size and first columns of the new_enf_df
print(len(enf_df1.sort_values('docket_number')))
print(enf_df1.head(5))

2029
      record_id docket_number      iid  \
1391       1463           100    100.0   
2907       3018           100    100.0   
1268       1337           100    100.0   
538         551         10100  10100.0   
2382       2478          1016   1016.0   

                                      unnamed:_3 docassociation unnamed:_5  \
1391  http://www.occ.gov/static/ots/enforcement/          94571       .pdf   
2907  http://www.occ.gov/static/ots/enforcement/          97102       .pdf   
1268  http://www.occ.gov/static/ots/enforcement/          93646       .pdf   
538   http://www.occ.gov/static/ots/enforcement/          93574       .pdf   
2382  http://www.occ.gov/static/ots/enforcement/          96124       .pdf   

                                             unnamed:_6  \
1391  http://www.occ.gov/static/ots/enforcement/9457...   
2907  http://www.occ.gov/static/ots/enforcement/9710...   
1268  http://www.occ.gov/static/ots/enforcement/9364...   
538   http://www.occ.gov/static/ots/e

In [20]:
#drops duplicates and adds 2 new colums to the new_enf_df

short_enf_df = enf_df1.drop_duplicates(subset=['docket_number'], keep='last')
#short_enf_df = short_enf_df.reindex(columns = new_enf_df.columns.tolist() + ['Number_of_Violations','Unique_Violations'])

#Gives the size and first columns of the short_enf_df
print(len(short_enf_df))
print(short_enf_df.head())

837
      record_id docket_number      iid  \
1268       1337           100    100.0   
538         551         10100  10100.0   
1865       1950          1016   1016.0   
2436       2533          1039      NaN   
1136       1202          1050      NaN   

                                      unnamed:_3 docassociation unnamed:_5  \
1268  http://www.occ.gov/static/ots/enforcement/          93646       .pdf   
538   http://www.occ.gov/static/ots/enforcement/          93574       .pdf   
1865  http://www.occ.gov/static/ots/enforcement/          94759       .pdf   
2436  http://www.occ.gov/static/ots/enforcement/          96246       .pdf   
1136  http://www.occ.gov/static/ots/enforcement/          93507       .pdf   

                                             unnamed:_6  \
1268  http://www.occ.gov/static/ots/enforcement/9364...   
538   http://www.occ.gov/static/ots/enforcement/9357...   
1865  http://www.occ.gov/static/ots/enforcement/9475...   
2436  http://www.occ.gov/static/ots/en

In [21]:
violation_count_df = enf_df1.pivot_table(index = ['docket_number'], aggfunc ='size')
vc_df = violation_count_df.reset_index()
vc_df.columns = ['docket_number', 'total_violations']
print(vc_df)

    docket_number  total_violations
0             100                 3
1           10100                 1
2            1016                 2
3            1039                 3
4            1050                 1
..            ...               ...
832           940                 1
833          9419                 3
834          9499                 1
835           966                 3
836         XXXXX                 2

[837 rows x 2 columns]


In [22]:
unique_violation_count_df = enf_df1.pivot_table(index = ['docket_number', "issue_date"], aggfunc ='size')
uvc_df = unique_violation_count_df.reset_index()
uvc_df.columns = ['docket_number', 'issue_date', 'violations']
print(uvc_df)

     docket_number  issue_date  violations
0              100   2/22/2006           1
1              100   4/17/2009           1
2              100   9/21/2004           1
3            10100  12/14/1994           1
4             1016    2/4/1999           2
...            ...         ...         ...
1573          9499   1/10/1996           1
1574           966  12/31/2009           1
1575           966    4/4/2008           1
1576           966   5/29/2009           1
1577         XXXXX   4/13/2011           2

[1578 rows x 3 columns]


In [23]:
final_uvc_df = uvc_df.pivot_table(index = ['docket_number'], aggfunc ='size')
fvc_df = violation_count_df.reset_index()
fvc_df.columns = ['docket_number', 'unique_violations']
print(fvc_df)

    docket_number  unique_violations
0             100                  3
1           10100                  1
2            1016                  2
3            1039                  3
4            1050                  1
..            ...                ...
832           940                  1
833          9419                  3
834          9499                  1
835           966                  3
836         XXXXX                  2

[837 rows x 2 columns]


In [24]:
violation_stats_df = vc_df.merge(fvc_df)
print(violation_stats_df)

    docket_number  total_violations  unique_violations
0             100                 3                  3
1           10100                 1                  1
2            1016                 2                  2
3            1039                 3                  3
4            1050                 1                  1
..            ...               ...                ...
832           940                 1                  1
833          9419                 3                  3
834          9499                 1                  1
835           966                 3                  3
836         XXXXX                 2                  2

[837 rows x 3 columns]


In [25]:
final_enf_df = short_enf_df.merge(violation_stats_df)
print(final_enf_df.head)

<bound method NDFrame.head of      record_id docket_number      iid  \
0         1337           100    100.0   
1          551         10100  10100.0   
2         1950          1016   1016.0   
3         2533          1039      NaN   
4         1202          1050      NaN   
..         ...           ...      ...   
832       1401           940    940.0   
833       1183          9419      NaN   
834       1015          9499      NaN   
835       3214           966    966.0   
836       3588         XXXXX      NaN   

                                     unnamed:_3 docassociation unnamed:_5  \
0    http://www.occ.gov/static/ots/enforcement/          93646       .pdf   
1    http://www.occ.gov/static/ots/enforcement/          93574       .pdf   
2    http://www.occ.gov/static/ots/enforcement/          94759       .pdf   
3    http://www.occ.gov/static/ots/enforcement/          96246       .pdf   
4    http://www.occ.gov/static/ots/enforcement/          93507       .pdf   
..             

In [26]:
df['docket'] = df['docket'].astype(str)
final_df = df.merge(final_enf_df, how='left', left_on='docket', right_on='docket_number')
small_final_df = final_df[['asset', 'chrtagnt', 'clcode', 'effdate', 'fdicregn', 'name', 'trust', 'total_violations', 'unique_violations']].copy()
print(small_final_df.head)

<bound method NDFrame.head of           asset chrtagnt  clcode     effdate  fdicregn  \
0           NaN    STATE      13  07/01/1981  New York   
1      712714.0    STATE      13  03/07/2002    Dallas   
2      338215.0      OCC       3  04/30/2018   Atlanta   
3      126549.0    STATE      13  10/16/1998   Atlanta   
4      144049.0    STATE      21  08/31/2022   Chicago   
...         ...      ...     ...         ...       ...   
27811       NaN    STATE      21  01/01/1984   Atlanta   
27812   62371.0    STATE      21  04/01/1998   Atlanta   
27813  178789.0    STATE      21  08/31/1996   Atlanta   
27814  454657.0    STATE      21  08/31/2022   Atlanta   
27815  200966.0    STATE      21  08/31/2022   Atlanta   

                                         name  trust  total_violations  \
0                   Northeast Bank of Sanford    NaN               NaN   
1                         Bank of Bentonville    1.0               NaN   
2      Progressive Bank, National Association    1.

In [27]:
small_final_df['total_violations'] = final_df['total_violations'].fillna(0)
small_final_df['unique_violations'] = final_df['unique_violations'].fillna(0)

In [28]:
no_violation_df = small_final_df[small_final_df.total_violations == 0]
violation_df = small_final_df[small_final_df.total_violations != 0]

print(no_violation_df.shape[0])
print(violation_df.shape[0])

26992
824


In [29]:
small_final_df.to_json('final.json', orient='records', lines=True)
no_violation_df.to_json('no_violation.json', orient='records', lines=True)
violation_df.to_json('violation.json', orient='records', lines=True)

In [30]:
import matplotlib

# plot a Stacked Bar Chart using matplotlib
small_final_df.plot(
  x = 'fdicregn', 
  kind = 'bar', 
  stacked = True, 
  title = 'Percentage Stacked Bar Graph', 
  mark_right = True) 

"""
df_total = df["Studied"] + df["Slept"] + df["Other"]
df_rel = df[df.columns[1:]].div(df_total, 0)*100
  
for n in df_rel:
    for i, (cs, ab, pc) in enumerate(zip(df.iloc[:, 1:].cumsum(1)[n], 
                                         df[n], df_rel[n])):
        plt.text(cs - ab / 2, i, str(np.round(pc, 1)) + '%', 
                 va = 'center', ha = 'center')
"""

'\ndf_total = df["Studied"] + df["Slept"] + df["Other"]\ndf_rel = df[df.columns[1:]].div(df_total, 0)*100\n  \nfor n in df_rel:\n    for i, (cs, ab, pc) in enumerate(zip(df.iloc[:, 1:].cumsum(1)[n], \n                                         df[n], df_rel[n])):\n        plt.text(cs - ab / 2, i, str(np.round(pc, 1)) + \'%\', \n                 va = \'center\', ha = \'center\')\n'

Error in callback <function flush_figures at 0x0000016947C10DC0> (for post_execute):


KeyboardInterrupt: 